In [3]:
import re
import spacy
import nltk
from nltk.corpus import wordnet
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, porter
from spacy.tokens.token import Token

import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import plotly_express as px
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_96730/2604194920.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Local files
from utils.melissa_utils import *
from utils.utils import *

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Some magic so that the notebook will reload the external python script file any time you edit and save the .py file;
%load_ext autoreload
%autoreload 2

In [24]:
dataset_path = Path("/Users/melloo21/Desktop/NUS Items/CS5228/Project/CS5228_project/dataset")
train_dataset = f"{dataset_path}/train.csv"
test_dataset = f"{dataset_path}/test.csv"

# READ
train_df = pd.read_csv(train_dataset)
test_df = pd.read_csv(test_dataset)

In [25]:
## Make Model Imputation
make_ls = train_df['make'].dropna().unique()
compiled_regex = compile_make_pattern(make_ls)

train_df = apply_make_extraction(train_df, compiled_regex)
train_df['make'] = train_df['make'].fillna(train_df['extracted_make'])

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         25000 non-null  int64  
 1   title              25000 non-null  object 
 2   make               25000 non-null  object 
 3   model              25000 non-null  object 
 4   description        24320 non-null  object 
 5   manufactured       24993 non-null  float64
 6   original_reg_date  255 non-null    object 
 7   reg_date           25000 non-null  object 
 8   type_of_vehicle    25000 non-null  object 
 9   category           25000 non-null  object 
 10  transmission       25000 non-null  object 
 11  curb_weight        24693 non-null  float64
 12  power              22360 non-null  float64
 13  fuel_type          5879 non-null   object 
 14  engine_cap         24404 non-null  float64
 15  no_of_owners       24982 non-null  float64
 16  depreciation       244

In [9]:
# Normalization :: cleanup of special characters/lower case/ uneven whitespace
rough_text_cleaner(train_df.model)
rough_text_cleaner(train_df.make)

 Initial number of unique : 799 
 Post cleaning : 799 
 Initial number of unique : 95 
 Post cleaning : 95 


0           land rover
1        mercedes_benz
2                honda
3               toyota
4                lexus
             ...      
24995    mercedes_benz
24996          bentley
24997    mercedes_benz
24998             audi
24999              bmw
Name: make, Length: 25000, dtype: object

In [26]:
## Task :: Imputation using same make and model and manufacture assumption
train_df["model_make"] = train_df.model + "_" + train_df.make

In [71]:
check_list = list()
for elem in train_df[train_df.curb_weight.isna()].model_make.unique():
    check_list.append(train_df[train_df.model_make ==  elem].curb_weight.isnull().all())
print(check_list.count(True))

44


In [65]:
elem = train_df[train_df.curb_weight.isna()].model_make.unique()[11]
print(elem)

boxster_porsche


In [74]:
train_df.columns

Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'indicative_price', 'price',
       'extracted_make', 'model_make'],
      dtype='object')

In [89]:
# Power, curbweight, fuel_type, engine_capacity , road_tax
display_feature_dist_by_ref_col(df=train_df,feature="engine_cap",ref="make")

Number of make unable to be inputted 9


make
aion                                                         [nan]
alfa romeo       [1779.0, 1368.0, 1995.0, 2891.0, 1962.0, 2198....
alpine                                                    [1798.0]
aston martin      [3982.0, 4735.0, 5935.0, 5204.0, 4282.0, 4280.0]
audi             [1395.0, 999.0, 4163.0, 1984.0, 1498.0, 2995.0...
austin                                       [998.0, 848.0, 948.0]
bentley          [3996.0, 3993.0, 5950.0, 6752.0, 5998.0, 6750....
bertone                                                   [1498.0]
bmw              [1499.0, 2996.0, 1998.0, 2998.0, 2979.0, 3999....
byd                                                          [nan]
cadillac                                          [3564.0, 6391.0]
camc                                    [9842.0, 11812.0, 11813.0]
chevrolet                 [1362.0, 1399.0, 1598.0, 1398.0, 4636.0]
chrysler                                          [3604.0, 2776.0]
citroen              [nan, 1199.0, 1598.0, 1560.0, 1997.0

In [87]:
display_feature_dist_by_ref_col(df=train_df,feature="curb_weight",ref="model_make")

Number of model_make unable to be inputted 44


model_make
(coe_land rover    [1760.0, 1845.0, 2397.0, 2512.0]
107_peugeot                                 [805.0]
116d_bmw                                   [1350.0]
116i_bmw                           [1310.0, 1340.0]
118i_bmw           [1315.0, 1320.0, 1520.0, 1300.0]
                                 ...               
zafira_opel                                [1538.0]
zk6117h_yutong          [11260.0, 11200.0, 11220.0]
zk6119h_yutong                            [11720.0]
zoe_renault                                [1480.0]
zs_mg                      [1610.0, 1532.0, 1534.0]
Name: curb_weight, Length: 809, dtype: object

In [80]:
display_feature_dist_by_ref_col(df=train_df,feature="power",ref="model_make")

Number of model make unable to be inputted 169


model_make
(coe_land rover             [nan, 171.0, 375.0]
107_peugeot                              [50.0]
116d_bmw                                 [85.0]
116i_bmw                          [100.0, 80.0]
118i_bmw           [125.0, 103.0, 105.0, 100.0]
                               ...             
zafira_opel                             [103.0]
zk6117h_yutong                            [nan]
zk6119h_yutong                            [nan]
zoe_renault                              [68.0]
zs_mg                             [99.0, 105.0]
Name: power, Length: 809, dtype: object

In [81]:
display_feature_dist_by_ref_col(df=train_df,feature="fuel_type",ref="model_make")

Number of model make unable to be inputted 459


model_make
(coe_land rover    [diesel, nan]
107_peugeot                [nan]
116d_bmw                [diesel]
116i_bmw                   [nan]
118i_bmw                   [nan]
                       ...      
zafira_opel                [nan]
zk6117h_yutong          [diesel]
zk6119h_yutong          [diesel]
zoe_renault           [electric]
zs_mg                 [electric]
Name: fuel_type, Length: 809, dtype: object

In [103]:
for index, value in train_df.groupby("model_make")["fuel_type"].nunique().items():
    if value >= 3:
        print(index)
        

amg_mercedes-benz
defender_land rover


In [107]:
train_df[train_df["model_make"]== "amg_mercedes-benz"].fuel_type

37                   NaN
38                diesel
81                   NaN
194      petrol-electric
255             electric
              ...       
24906                NaN
24920                NaN
24956                NaN
24983    petrol-electric
24984    petrol-electric
Name: fuel_type, Length: 809, dtype: object

In [44]:
def fill_column_b_with_mean(df, column_a, column_b):
    # Calculate the mean of column B grouped by column A
    values = df.groupby(column_a)[column_b].agg(["mean"]).rename(columns={"mean": column_b})[column_b]
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # If column B is NaN, fill it with the mean value of column B for the corresponding value in column A
        if pd.isna(row[column_b]):
            df.at[index, column_b] = values[row[column_a]]
    
    return df

In [38]:
train_df.groupby(ref_col)[feature].mean()

model_make
(coe_land rover     2128.500000
107_peugeot          805.000000
116d_bmw            1350.000000
116i_bmw            1330.588235
118i_bmw            1325.408163
                       ...     
zafira_opel         1538.000000
zk6117h_yutong     11226.666667
zk6119h_yutong     11720.000000
zoe_renault         1480.000000
zs_mg               1544.461538
Name: curb_weight, Length: 809, dtype: float64

In [43]:
train_df.groupby(ref_col)[feature].agg(["mean"]).rename(columns={"mean": feature})[feature]

model_make
(coe_land rover     2128.500000
107_peugeot          805.000000
116d_bmw            1350.000000
116i_bmw            1330.588235
118i_bmw            1325.408163
                       ...     
zafira_opel         1538.000000
zk6117h_yutong     11226.666667
zk6119h_yutong     11720.000000
zoe_renault         1480.000000
zs_mg               1544.461538
Name: curb_weight, Length: 809, dtype: float64

In [22]:
feature = "curb_weight"
ref_col = "model_make"
elem = train_df[train_df[feature].isna()].model_make.unique()[11]
print(elem)
train_df[train_df[ref_col]== elem][feature]

testarossa_ferrari


3049   NaN
Name: curb_weight, dtype: float64

In [45]:
new_df = fill_column_b_with_mean(train_df,ref_col,feature)

In [18]:
new_df[new_df[ref_col]== elem][feature]

1121     1425.000000
1144     1425.000000
1246     1455.000000
1310     1410.923077
3470     1455.000000
3535     1380.000000
3687     1410.923077
3805     1310.000000
4371     1400.000000
4535     1393.000000
4681     1440.000000
5380     1340.000000
5706     1440.000000
6199     1415.000000
6662     1375.000000
7324     1415.000000
7407     1340.000000
7602     1425.000000
7670     1430.000000
8310     1410.923077
8908     1415.000000
9219     1470.000000
9252     1410.923077
9511     1425.000000
10005    1420.000000
10485    1425.000000
10682    1440.000000
11700    1340.000000
12304    1395.000000
12307    1425.000000
12381    1375.000000
13373    1440.000000
13493    1355.000000
13643    1425.000000
14890    1455.000000
15487    1410.923077
15907    1395.000000
16073    1395.000000
16614    1455.000000
16808    1455.000000
17023    1425.000000
17307    1425.000000
17809    1340.000000
18098    1415.000000
18483    1365.000000
18648    1410.923077
18676    1455.000000
18749    1415